In [1]:
import pandas as pd
import numpy as np
import requests

<H1>Exercise 1</H1>

In [3]:
#loadthe provided CSV (sample_data.csv) file into a Pandas DataFrame.

data=pd.read_csv('sample_data.csv')


C:\Users\mj-98\AppData\Local\Temp\ipykernel_11808\4109198742.py:3: DtypeWarning: Columns (8,25,30,31,32,33,34,36,37,38,41,42,43,44,45,46,47,48,54,55,57,58,60,62,63,64,65,67,69,70,71,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('sample_data.csv')


In [4]:
#2.	Replace all nan values in the DataFrame with empty cells. 
data.fillna("", inplace=True)
#3.	Remove any trailing spaces from the country names in the GeoAreaName column.
data["GeoAreaName"]=data.GeoAreaName.str.strip()

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236476 entries, 0 to 236475
Data columns (total 75 columns):
 #   Column                                    Non-Null Count   Dtype 
---  ------                                    --------------   ----- 
 0   Goal                                      236476 non-null  int64 
 1   Target                                    236476 non-null  object
 2   Indicator                                 236476 non-null  object
 3   SeriesCode                                236476 non-null  object
 4   SeriesDescription                         236476 non-null  object
 5   GeoAreaCode                               236476 non-null  int64 
 6   GeoAreaName                               236476 non-null  object
 7   TimePeriod                                236476 non-null  int64 
 8   Value                                     236476 non-null  object
 9   Time_Detail                               236476 non-null  object
 10  TimeCoverage                    

In [4]:
# 4- Add two new columns, ISO and GeoAreaName_ar, using the information from the attached Excel file (country_info.xlsx).
data_country=pd.read_excel("country_info.xlsx")

country_dict={}
country_dict_arabic={}
count=0

#I created a function here to add the ISOs and the Arabic names to
#seperate dictionaries with the key being the country so that I can use
#the dictionaries for the mapping function below.
for country in data_country.Country:
    country_dict[country]=data_country.ISO3[count]
    country_dict_arabic[country]=data_country["Arabic Name"][count]
    count=count+1

#I created a function that compares the country name found in the dataframe
#with the key in the dictionary. If they match, I would then return the value
# that will be added to the main dataframe below.

def country_mapping(country):
    for compare in country_dict.keys():
        if(country==compare):
            return country_dict.get(compare)

def country_mapping_arabic(country):
    for compare in country_dict_arabic.keys():
        if(country==compare):
            return country_dict_arabic.get(compare)


data["ISO"]=data.GeoAreaName.map(country_mapping)
data["GeoAreaName_ar"]=data.GeoAreaName.map(country_mapping_arabic)

data.head(5)

,Goal,Target,Indicator,SeriesCode,SeriesDescription,GeoAreaCode,GeoAreaName,TimePeriod,Value,Time_Detail,...,Cause of death,Illicit Financial Flows,Parliamentary committees,Population Group,Service Attribute,Tariff regime (status),Type of OFDI scheme,Type of speed,ISO,GeoAreaName_ar
0,5,5.3,5.3.1,SP_DYN_MRBF18,Proportion of women aged 20-24 years who were ...,262,Djibouti,2019,6.5,2019,...,,,,,,,,,DJI,جيبوتي
1,5,5.3,5.3.2,SH_STA_FGMS,Proportion of girls and women aged 15-49 years...,262,Djibouti,2019,90.1,2019,...,,,,,,,,,DJI,جيبوتي
2,5,5.3,5.3.1,SP_DYN_MRBF15,Proportion of women aged 20-24 years who were ...,262,Djibouti,2019,1.4,2019,...,,,,,,,,,DJI,جيبوتي
3,5,5.5,5.5.2,IC_GEN_MGTL,Proportion of women in managerial positions - ...,262,Djibouti,2017,12.31,2017,...,,,,,,,,,DJI,جيبوتي
4,5,5.5,5.5.1,SG_GEN_PARLN,Number of seats held by women in national parl...,262,Djibouti,2000,0.0,2000,...,,,,,,,,,DJI,جيبوتي


In [5]:
#5.	Filter out any rows that don't match an ISO code from the Excel file and save these rows in a separate DataFrame, 
#keeping the original row indices

#the rows that won't match will be the ones that have an ISO that is empty.
def filter_not_matching(data):
    filtered_data=data[data[["ISO"]].isnull().any(axis=1)]
    return filtered_data

filtered_data_ISO=filter_not_matching(data)

filtered_dataframe=pd.DataFrame(filtered_data_ISO) #the indices are kept the same. 
print(filtered_dataframe)

# I would need to reset the indices for it to be reset
# reset_indices=filtered_dataframe.reset_index(drop=True)
# print(reset_indices)


#I also noticed that the rows that don't have the ISO are: palestine and Syria. They do have an ISO that belongs to them
#So, we can possibly go over the GeoAreaName and make sure that the countries of the same name are correct.
#For instance, Syria would be changed to Syrian Arab Republic.  

        Goal Target Indicator     SeriesCode  \
18590      2    2.1     2.1.2  AG_PRD_FIESMS   
18591      2    2.1     2.1.2  AG_PRD_FIESMS   
18604      2    2.1     2.1.2  AG_PRD_FIESMS   
18607      2    2.1     2.1.2  AG_PRD_FIESMS   
188701     5    5.5     5.5.1   SG_GEN_PARLN   
188713     5    5.5     5.5.1   SG_GEN_PARLN   

                                        SeriesDescription  GeoAreaCode  \
18590   Prevalence of moderate or severe food insecuri...          275   
18591   Prevalence of moderate or severe food insecuri...          275   
18604   Prevalence of moderate or severe food insecuri...          275   
18607   Prevalence of moderate or severe food insecuri...          275   
188701  Number of seats held by women in national parl...          760   
188713  Number of seats held by women in national parl...          760   

       GeoAreaName  TimePeriod Value Time_Detail  ... Cause of death  \
18590    Palestine        2018  28.9   2017-2019  ...                  


In [6]:
#6.Drop any columns in the original DataFrame that have no data.

#if the columns still had the null values as opposed to them simply being empty
#we could have used the built in function: data.dropna(axis=1,how='all',inplace=True)
#since we have the "" in the empty columns, we will go over the columns
#and remove the column if it has the empty string in it
for col in data.columns:
    if data[col].eq("").all() :
        data.drop(col, axis=1,inplace=True)

In [7]:
data.shape
data.head()

,Goal,Target,Indicator,SeriesCode,SeriesDescription,GeoAreaCode,GeoAreaName,TimePeriod,Value,Time_Detail,...,Bioclimatic belt,Land cover,Cause of death,Parliamentary committees,Population Group,Service Attribute,Type of OFDI scheme,Type of speed,ISO,GeoAreaName_ar
0,5,5.3,5.3.1,SP_DYN_MRBF18,Proportion of women aged 20-24 years who were ...,262,Djibouti,2019,6.5,2019,...,,,,,,,,,DJI,جيبوتي
1,5,5.3,5.3.2,SH_STA_FGMS,Proportion of girls and women aged 15-49 years...,262,Djibouti,2019,90.1,2019,...,,,,,,,,,DJI,جيبوتي
2,5,5.3,5.3.1,SP_DYN_MRBF15,Proportion of women aged 20-24 years who were ...,262,Djibouti,2019,1.4,2019,...,,,,,,,,,DJI,جيبوتي
3,5,5.5,5.5.2,IC_GEN_MGTL,Proportion of women in managerial positions - ...,262,Djibouti,2017,12.31,2017,...,,,,,,,,,DJI,جيبوتي
4,5,5.5,5.5.1,SG_GEN_PARLN,Number of seats held by women in national parl...,262,Djibouti,2000,0.0,2000,...,,,,,,,,,DJI,جيبوتي


In [8]:
# 7.	Save the cleaned DataFrame into a JSON file entitled output_data.json.
data.to_json("output_data.json")
#I noticed that we still have some empty data in certain columns.
#we need to deal with them as well in order to have a cleaner dataset
#we might remove them if they are not alot, revist the data collection/data entry to collect/check for information
#or we might try to fill them with a value. 
#For instance, we might replace the "" of the Bioclimatic belt with "none".

<H1>Exercise 2</H1>

In [17]:
# 1.Load data from from the ‘data ’ object in the API https://unstats.un.org/SDGAPI/v1/sdg/Series/Data?page=100
#for the pages 100 to 105 into a panda DataFrame named df1.
url='https://unstats.un.org/SDGAPI/v1/sdg/Series/Data?page='
df1=pd.DataFrame()
#For each page, specify the URL and use the request to get the 'data' object
#and add it to a dataframe
for i in range(100,106):
    url1=url+str(i)
    testing=requests.get(url1)
    
    if testing.status_code==200: #if the request is successful
        try:
            get_data=testing.json()['data'] #get the data if it exists
            if get_data:
                df= pd.DataFrame(get_data) #add it to the dataframe
                df1=pd.concat([df1,df],ignore_index=True)
            else:
                print("Data not found") #if data is not found
        except testing.JSONDecodeError: #if the request failed
                print("Unable to parse")
    else:
        print("Request failed")
    
print(df1.head(5))


  goal target indicator       series  \
0  [1]  [1.1]   [1.1.1]  SI_POV_DAY1   
1  [1]  [1.1]   [1.1.1]  SI_POV_DAY1   
2  [1]  [1.1]   [1.1.1]  SI_POV_DAY1   
3  [1]  [1.1]   [1.1.1]  SI_POV_DAY1   
4  [1]  [1.1]   [1.1.1]  SI_POV_DAY1   

                                   seriesDescription seriesCount geoAreaCode  \
0  Proportion of population below international p...       10284         515   
1  Proportion of population below international p...       10284         515   
2  Proportion of population below international p...       10284         515   
3  Proportion of population below international p...       10284         515   
4  Proportion of population below international p...       10284         515   

          geoAreaName  timePeriodStart value  ... time_detail timeCoverage  \
0  Developing regions           1990.0  48.9  ...        None         None   
1  Developing regions           1991.0  47.9  ...        None         None   
2  Developing regions           1992.0  46.5

In [18]:
# 2.Load data from the ‘data ’ object in the API https://unstats.un.org/SDGAPI/v1/sdg/Series/Data?page=117430 from page 117430 to page 117435 into a pandas DataFrame named df2.
url='https://unstats.un.org/SDGAPI/v1/sdg/Series/Data?page='
df2=pd.DataFrame()
for i in range(117430,117436):
    url1=url+str(i)
    testing=requests.get(url1)
    
    if testing.status_code==200:
        try:
            get_data=testing.json()['data']
            if get_data:
                df= pd.DataFrame(get_data)
                df2=pd.concat([df2,df],ignore_index=True)
            else:
                print("Data not found")
        except testing.JSONDecodeError:
                print("Unable to parse")
    else:
        print("Request failed")
    
print(df2.head(5))

Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Empty DataFrame
Columns: []
Index: []


In [19]:
#3- Merge df1 and df2 into a single DataFrame df3.
#We can always check and see if the two dataframes have the same #of columns and rows
#and if they have the same columns before performing the merge
print("Info of df1",df1.info())
print("\nInfo of df2",df2.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   goal               150 non-null    object 
 1   target             150 non-null    object 
 2   indicator          150 non-null    object 
 3   series             150 non-null    object 
 4   seriesDescription  150 non-null    object 
 5   seriesCount        150 non-null    object 
 6   geoAreaCode        150 non-null    object 
 7   geoAreaName        150 non-null    object 
 8   timePeriodStart    150 non-null    float64
 9   value              150 non-null    object 
 10  valueType          150 non-null    object 
 11  time_detail        0 non-null      object 
 12  timeCoverage       0 non-null      object 
 13  upperBound         0 non-null      object 
 14  lowerBound         0 non-null      object 
 15  basePeriod         150 non-null    object 
 16  source             150 non

In [20]:
#if all is well, then we can concatenate them
df3=pd.concat([df1,df2],ignore_index=True)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   goal               150 non-null    object 
 1   target             150 non-null    object 
 2   indicator          150 non-null    object 
 3   series             150 non-null    object 
 4   seriesDescription  150 non-null    object 
 5   seriesCount        150 non-null    object 
 6   geoAreaCode        150 non-null    object 
 7   geoAreaName        150 non-null    object 
 8   timePeriodStart    150 non-null    float64
 9   value              150 non-null    object 
 10  valueType          150 non-null    object 
 11  time_detail        0 non-null      object 
 12  timeCoverage       0 non-null      object 
 13  upperBound         0 non-null      object 
 14  lowerBound         0 non-null      object 
 15  basePeriod         150 non-null    object 
 16  source             150 non

In [21]:
# 4.Convert ‘Value’ column in df3 from a string to float.
df3["value"]=df["value"].astype("float")
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   goal               150 non-null    object 
 1   target             150 non-null    object 
 2   indicator          150 non-null    object 
 3   series             150 non-null    object 
 4   seriesDescription  150 non-null    object 
 5   seriesCount        150 non-null    object 
 6   geoAreaCode        150 non-null    object 
 7   geoAreaName        150 non-null    object 
 8   timePeriodStart    150 non-null    float64
 9   value              25 non-null     float64
 10  valueType          150 non-null    object 
 11  time_detail        0 non-null      object 
 12  timeCoverage       0 non-null      object 
 13  upperBound         0 non-null      object 
 14  lowerBound         0 non-null      object 
 15  basePeriod         150 non-null    object 
 16  source             150 non

In [22]:
# 5.Extract a list of unique sources from the ‘source’ column in df3. 
unique_sources=df3.source.unique()
df3.source.unique()

array(['Poverty and Inequality Portal, World Bank'], dtype=object)

In [23]:
#6.	Save the final DataFrame as output_data.csv.
df3.to_csv("output_data.csv")